In [134]:
from __future__ import division, absolute_import, print_function
%load_ext autoreload
%autoreload 2

import os 
#os.chdir(r"D:\data\ortoCC\design-with-alignments\mason_small")
print(os.getcwd()) 
    
import hepran
import hepran.bzipscore as bz
import hepran.bcipa as bc
import hepran.utils as u
import hepran.registers as r

import hepran.agadir as ag

import pandas as pd
import numpy as np
import seaborn as sns
    
%matplotlib inline

import matplotlib as mpl
import matplotlib.pylab as plt

from  score_utils import *
    
CCO = os.environ.get("CCO",r"C:\Projekti_KI\ortoCC\CoiledCoilOrtho")

set_dir = "D:\data\ortoCC\design-with-alignments\4heptade-7\!OUT_bcf"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
d:\data\ortoCC\fine_tune_scoring_function


In [98]:
from interactive_set_plot import *

import bokeh
import bokeh.resources
import bokeh.plotting as bp
from bokeh.models import HoverTool
from IPython.core.display import display, HTML
import hepran
bp.output_notebook()

import sklearn
from sklearn import linear_model
from sklearn.externals import joblib

from IPython.display import display, HTML

Loading BokehJS ...

In [156]:
df_dna=pd.read_excel('data/all.xlsx', index_col=[0,1])

In [167]:
df = pd.read_excel('data/round0_fiting.xlsx',index_col=[0,1])
df.head(5);
sc = pd.read_excel('data/mason_both.ln_mean_RD.score.xlsx'.format(**locals()),index_col=[0,1] )
#if sfunc in "iCipa_basic_ends_rep,iCipa_basic_rep,iCipa_basic_ends,iCipa_basic,ln_med_RD,ln_mean_RD".split(','):
#    sc['score']=sc['score']*-1
sc = sc.join(df['Tm on_target seq1 seq2'.split()], lsuffix='_orig')

sc['ln_mean_RD'] = sc['score']
sc['IDs'] = sc.index
sc = sc.apply(get_formated_seq, axis=1)
#sc = sc.reset_index()
fit_type = 'LinearRegression'
N_iter = 0

In [168]:
sc = sc.join(df_dna['bcnum'.split()], lsuffix='_orig')

In [169]:
sc['weights']=1
#sc.weights = sc.bcnum


In [170]:
lm, R2 = make_model('Tm', ['ln_mean_RD'], sc.query('ln_mean_RD > -4'), fit_type)

In [171]:

sc['score'] = lm.predict(sc[['ln_mean_RD']])

In [172]:
tooltips = [
    ('ID1, ID2', '@IDs'),
    ('Tm', '@Tm'),
    ('score', '@score'),
    ('ln_mean_RD', '@ln_mean_RD'),
    ('bcnum', '@bcnum'),
    ('seq1', '@seq1_disp{safe}'),
    ('seq2', '@seq2_disp{safe}'),
]

p = draw_scatter_interactive('Tm','score',sc, y_range=(0,80),                                
                                 title='Tm vs predicted Tm based on ln_med_RD', save_to_file=False, tooltips=tooltips)


bp.show(p)


In [165]:
lm.coef_, lm.intercept_

(array([21.14190406]), -63.22241667352923)

In [166]:
fit_metric = get_FIT_dataframe(sc, lm, N_iter, fit_type, N_feat=len(lm.coef_)+1, N_samples=len(sc))
display(fit_metric)

corrR  R2_score       RMSE  med_abs_err  \
N_iter fit_type         fit_class                                               
0      LinearRegression all        0.740502  0.583590  11.433620     2.979692   
                        Tm>25      0.806220  0.289274  11.885252     3.227800   
                        Tm>55      0.692887 -2.747539  12.713342     3.422874   
                        on_target  0.419612 -1.330082  11.205338     3.124382   

                                   explained_var   Baysian_IC    Akaike_IC  \
N_iter fit_type         fit_class                                            
0      LinearRegression all             0.546373  1336.699240  2854.265797   
                        Tm>25           0.499512   712.896364  1410.712152   
                        Tm>55          -1.893479   107.697545   165.620726   
                        on_target      -1.653289    82.869665   125.685907   

                                  N_samples N_feat  
N_iter fit_type         fit_class                   
0      LinearRegression all             272      2  
                        Tm>25           142      2  
                        Tm>55            20      2  
                        on_target        16      2

In [174]:
fit_metric = get_FIT_dataframe(sc, lm, N_iter, fit_type, N_feat=len(lm.coef_)+1, N_samples=len(sc))
display(fit_metric)

corrR  R2_score       RMSE  med_abs_err  \
N_iter fit_type         fit_class                                               
0      LinearRegression all        0.740502  0.548344  11.348429     2.959565   
                        Tm>25      0.806220  0.260923  11.538235     3.099885   
                        Tm>55      0.692887 -2.474757  12.331073     3.143505   
                        on_target  0.419612 -1.614645  10.756302     3.011345   

                                   explained_var   Baysian_IC    Akaike_IC  \
N_iter fit_type         fit_class                                            
0      LinearRegression all             0.548344  1332.630734  2850.197292   
                        Tm>25           0.547352   704.480877  1402.296665   
                        Tm>55          -1.520164   106.476358   164.399539   
                        on_target      -1.392183    81.560915   124.377157   

                                  N_samples N_feat  
N_iter fit_type         fit_class                   
0      LinearRegression all             272      2  
                        Tm>25           142      2  
                        Tm>55            20      2  
                        on_target        16      2

In [136]:
ln_mean_RD_to_Tm(4.495)

31.984694761050676

In [19]:
core_p, es_p = r.get_core_and_es_pairs('DKNAALKAENAALEYEIAALEAEIAALEG', 'DEIAALEAKNAALKYKIAALKAKIAALKG', 'f','f')
print(core_p)
r.pairs_counter(r.get_vertical_triplets(es_p), sort=False)

[('N', 'I'), ('N', 'N'), ('I', 'I'), ('I', 'I')]


Counter({'EEE': 2,
         'EEK': 1,
         'EKE': 1,
         'EKK': 2,
         'KEE': 2,
         'KEK': 1,
         'KKE': 1,
         'KKK': 2})

In [16]:
#core_p, es_p = r.get_core_and_es_pairs('DKNAALKAENAALEYEIAALEAEIAALEG', 'DEIAALEAKNAALKYKIAALKAKIAALKG', 'f','f')





df = df.apply(get_CC_features, axis=1)

In [17]:
df.columns

Index([               u'seq1',                u'seq2',                  u'Tm',
                 u'on_target',             u'avg_HP1',             u'avg_HP2',
                 u'avg_HP_12',                 u'HP1',                 u'HP2',
                     u'HP_12',          u'nterm_c_NN',          u'nterm_c_IN',
                u'nterm_c_II',          u'cterm_c_NN',          u'cterm_c_IN',
                u'cterm_c_II',                u'c_NN',                u'c_IN',
                      u'c_II',               u'es_EE',               u'es_EK',
                     u'es_KK',   u'interface_charge1',   u'interface_charge2',
       u'interface_repulsion',               u'cv_NN',               u'cv_II',
                     u'cv_NI',               u'cv_IN',              u'esv_KK',
                    u'esv_EE',              u'esv_EK',              u'esv_KE',
                   u'cvt_NNI',             u'cvt_IIN',             u'cvt_NNN',
                   u'cvt_NIN',             u'cvt_INN

In [162]:
df.head(50)

seq1                           seq2  Tm  \
ID1 ID2                                                                     
O1  O1   DKNAALKAENAALEYEIAALEAEIAALEG  DKNAALKAENAALEYEIAALEAEIAALEG  -2   
    O2   DKNAALKAENAALEYEIAALEAEIAALEG  DENAALEAKNAALKYKIAALKAKIAALKG  72   
O2  O2   DENAALEAKNAALKYKIAALKAKIAALKG  DENAALEAKNAALKYKIAALKAKIAALKG  34   
O1  O3   DKNAALKAENAALEYEIAALEAEIAALEG  DEIAALEAKIAALKYENAALEAENAALEG   3   
O2  O3   DENAALEAKNAALKYKIAALKAKIAALKG  DEIAALEAKIAALKYENAALEAENAALEG  27   
O3  O3   DEIAALEAKIAALKYENAALEAENAALEG  DEIAALEAKIAALKYENAALEAENAALEG  13   
O1  O4   DKNAALKAENAALEYEIAALEAEIAALEG  DKIAALKAEIAALEYKNAALKAKNAALKG  20   
O2  O4   DENAALEAKNAALKYKIAALKAKIAALKG  DKIAALKAEIAALEYKNAALKAKNAALKG  21   
O3  O4   DEIAALEAKIAALKYENAALEAENAALEG  DKIAALKAEIAALEYKNAALKAKNAALKG  68   
O4  O4   DKIAALKAEIAALEYKNAALKAKNAALKG  DKIAALKAEIAALEYKNAALKAKNAALKG  21   
O1  O5   DKNAALKAENAALEYEIAALEAEIAALEG  DENAALEAEIAALEYKNAALKAEIAALEG  21   
O2  O5   DENAALEAKNAALKYKIAALKAKIAALKG  DENAALEAEIAALEYKNAALKAEIAALEG  30   
O3  O5   DEIAALEAKIAALKYENAALEAENAALEG  DENAALEAEIAALEYKNAALKAEIAALEG  30   
O4  O5   DKIAALKAEIAALEYKNAALKAKNAALKG  DENAALEAEIAALEYKNAALKAEIAALEG  36   
O5  O5   DENAALEAEIAALEYKNAALKAEIAALEG  DENAALEAEIAALEYKNAALKAEIAALEG  17   
O1  O6   DKNAALKAENAALEYEIAALEAEIAALEG  DKNAALKAKIAALKYENAALEAKIAALKG  23   
O2  O6   DENAALEAKNAALKYKIAALKAKIAALKG  DKNAALKAKIAALKYENAALEAKIAALKG  30   
O3  O6   DEIAALEAKIAALKYENAALEAENAALEG  DKNAALKAKIAALKYENAALEAKIAALKG  33   
O4  O6   DKIAALKAEIAALEYKNAALKAKNAALKG  DKNAALKAKIAALKYENAALEAKIAALKG  38   
O5  O6   DENAALEAEIAALEYKNAALKAEIAALEG  DKNAALKAKIAALKYENAALEAKIAALKG  71   
O6  O6   DKNAALKAKIAALKYENAALEAKIAALKG  DKNAALKAKIAALKYENAALEAKIAALKG  23   
O1  O7   DKNAALKAENAALEYEIAALEAEIAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  34   
O2  O7   DENAALEAKNAALKYKIAALKAKIAALKG  DKIAALKAKNAALKYKIAALKAENAALEG  15   
O3  O7   DEIAALEAKIAALKYENAALEAENAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  18   
O4  O7   DKIAALKAEIAALEYKNAALKAKNAALKG  DKIAALKAKNAALKYKIAALKAENAALEG  16   
O5  O7   DENAALEAEIAALEYKNAALKAEIAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  35   
O6  O7   DKNAALKAKIAALKYENAALEAKIAALKG  DKIAALKAKNAALKYKIAALKAENAALEG  19   
O7  O7   DKIAALKAKNAALKYKIAALKAENAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  -1   
O1  O8   DKNAALKAENAALEYEIAALEAEIAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  -3   
O2  O8   DENAALEAKNAALKYKIAALKAKIAALKG  DEIAALEAENAALEYEIAALEAKNAALKG  51   
O3  O8   DEIAALEAKIAALKYENAALEAENAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  20   
O4  O8   DKIAALKAEIAALEYKNAALKAKNAALKG  DEIAALEAENAALEYEIAALEAKNAALKG  22   
O5  O8   DENAALEAEIAALEYKNAALKAEIAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  26   
O6  O8   DKNAALKAKIAALKYENAALEAKIAALKG  DEIAALEAENAALEYEIAALEAKNAALKG  48   
O7  O8   DKIAALKAKNAALKYKIAALKAENAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  68   
O8  O8   DEIAALEAENAALEYEIAALEAKNAALKG  DEIAALEAENAALEYEIAALEAKNAALKG   2   
O1  O9   DKNAALKAENAALEYEIAALEAEIAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  12   
O2  O9   DENAALEAKNAALKYKIAALKAKIAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  56   
O3  O9   DEIAALEAKIAALKYENAALEAENAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  18   
O4  O9   DKIAALKAEIAALEYKNAALKAKNAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  28   
O5  O9   DENAALEAEIAALEYKNAALKAEIAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  11   
O6  O9   DKNAALKAKIAALKYENAALEAKIAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  40   
O7  O9   DKIAALKAKNAALKYKIAALKAENAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  17   
O8  O9   DEIAALEAENAALEYEIAALEAKNAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  22   
O9  O9   DENAALEAKNAALKYEIAALEAEIAALEG  DENAALEAKNAALKYEIAALEAEIAALEG   6   
O1  O10  DKNAALKAENAALEYEIAALEAEIAALEG  DKNAALKAENAALEYKIAALKAKIAALKG  42   
O10 O2   DKNAALKAENAALEYKIAALKAKIAALKG  DENAALEAKNAALKYKIAALKAKIAALKG  46   
    O3   DKNAALKAENAALEYKIAALKAKIAALKG  DEIAALEAKIAALKYENAALEAENAALEG  10   
    O4   DKNAALKAENAALEYKIAALKAKIAALKG  DKIAALKAEIAALEYKNAALKAKNAALKG  32   
    O5   DKNAALKAENAALEYKIAALKAKIAALKG  DENAALEAEIAALEYKNAALKAEIAALEG  48   

         on_target   avg_HP1 

In [163]:
df.to_excel('data/round0_fiting.xlsx')